In [322]:
import numpy as np
from collections import defaultdict as ddict

class Sudoku:
    """
    Examples:
        game = Sudoku(9).random()
        game
        game.solve(maxiter=1000)
    """
    xinterval = "  "
    yinterval = "\n"

    def __init__(self, width):
        self.width = width
        self.clear()
        self.maxlen = len(str(width))
        if self.maxlen >= 2:
            self.yinterval = "\n\n"
            self.xinterval = "   "
        self.elementrange = range(1, width+1, 1)
        self.baseset = set(self.elementrange)
        self.xrange = range(0, width, 1)
        self.yrange = range(0, width, 1)
        self.enumerate = lambda x: enumerate(x, 0)

    def random(self):
        width = self.width
        verify = False
        while not verify:
            self.clear()
            ij_pairs = np.random.randint(0, width, (width, 2), dtype=int)
            if len(np.unique(ij_pairs, axis=0)) != width:
                continue
            element_list = np.random.choice(self.elementrange, width, replace=False)
            for (i, j), element in zip(ij_pairs, element_list):
                self.canvas[i, j] = element
            verify = self.is_valid
        return self

    def clear(self):
        self.canvas = np.zeros((self.width, self.width), dtype=int)
        self.badnodes = ddict(set)

    def __str__(self):
        out = "\n".join(
        self.xinterval.join(f"{element:>{self.maxlen}}" if element else (" "*self.maxlen) for element in row) for row in self.canvas
        )
        return out

    def __repr__(self):
        heading = " "*self.maxlen + self.xinterval + self.xinterval.join(f"{x:>{self.maxlen}}" for x in self.xrange) + "\n\n"
        out = self.yinterval.join(
            f"{i:>{self.maxlen}}{self.xinterval}{row}" for i,row in self.enumerate(self.__str__().split("\n"))
            )
        return heading + out

    def place(i, j, element):
        return None

    def row(self, i):
        return self.canvas[i, :]

    def col(self, i):
        return self.canvas[:, i]

    def guess(self):
        canvas = self.canvas
        baseset = self.baseset
        badnodes = self.badnodes
        query = self.query
        graph = {}
        for i in self.yrange:
            for j in self.xrange:
                if not (element := canvas[i, j]):
                    graph[i, j] = query(i, j)
        return graph

    def query(self, i, j):
        available = self.baseset - (set(self.row(i)) | set(self.col(j)))
        badnodes = self.badnodes
        if (i, j) in badnodes:
            available -= badnodes[i, j]
        return available

    def solve(self, maxiter=1000):
        from time import perf_counter as tt
        canvas = self.canvas
        node_history = []
        tree_history = [self.guess()]
        verify_tree = self.verify_guess
        n_cycle = 0
        depth = 0
        filename = "Sudoku.log"
        with open(filename, "w") as log:
            _ = log.write(f"Cycle = {n_cycle}\n{repr(self)}\n--------\n")
            t0 = tt()
            # === START Depth First Search
            ij = None
            while (not self.check(verbose=False)) and (n_cycle < maxiter):
                n_cycle += 1
                tree = tree_history[depth]
                if verify_tree(tree):
                    # Forward Propagation
                    n_singular = 0
                    for ij, childs in tree.items():
                        if len(childs) == 1:
                            for child in childs:
                                canvas[ij] = child
                                node_history.append(ij)
                                n_singular += 1
                                break
                            break  # Travel only one step per cycle
                    if not n_singular:
                        for ij, childs in tree.items():
                            for child in childs:
                                canvas[ij] = child
                                node_history.append(ij)
                                break
                            break
                    tree_history.append(self.guess())
                    depth += 1
                    _ = log.write(f"Cycle = {n_cycle}\n{repr(self)}\n--------\n")
                else:
                    # Backward Propagation
                    # The latest node if not with respect to current tree,
                    # it is with respect to the previous tree.
                    # Therefore we discard the current tree.
                    _ = log.write(
                        f"Backward Propagation\n{depth = }\n{ij = }\n{repr(self)}\n--------\n"
                        f"{tree = }\n--------\n")
                    _ = tree_history.pop()
                    depth -= 1
                    tree = tree_history[depth]
                    ij = node_history.pop()
                    # Retrieve value and undo last step
                    value = canvas[ij]
                    canvas[ij] = 0
                    tree[ij].discard(value)
                    _ = log.write(f"{depth = }\n{ij = }\nTree = {tree}\n--------\n")
                    # If the branch is empty after discard,
                    # This indicate that the current state is unsolvable.
                    # So we need to discard this tree also.
                    # Checking whether this tree is solvable or not is done in the begining,
                    # so nothing is being done after this.
                    # But if there is only one possibility left after discarding,
                    # This will fall into the first case of Forward Propagation.
                    _ = log.write(f"END Backward Propagation\n--------\n")
            # === END Depth First Search
            t1 = tt()
            _ = log.write(f"Finished. \nCPU time = {t1-t0:.5f} s\n")
            _ = log.write(f"{self.check(verbose=False) = }")
            print(f"Finished. \nCPU time = {t1-t0:.5f} s\nCycle = {n_cycle} >>> {filename}")
        return self

    @property
    def is_valid(self):
        return not any(True for x in self.guess().values() if not x)

    def verify_guess(self, guess):
        return not any(True for x in guess.values() if not x)

    def check(self, verbose=True):
        if 0 in self.canvas:
            return False
        canvas = self.canvas
        width = self.width
        for i in self.yrange:
            if len(set(self.row(i))) != width:
                if verbose:
                    print(f"{i} row FAIL")
                return False
        for i in self.xrange:
            if len(set(self.col(i))) != width:
                if verbose:
                    print(f"{i} col FAIL")
                return False
        return True

In [343]:
game = Sudoku(9).random()
game
game.solve(maxiter=1000)

   0  1  2  3  4  5  6  7  8

0                           
1  9                        
2  4                        
3                       7   
4              8  6         
5                       5   
6  1              3         
7                    2      
8                           

Finished. 
CPU time = 0.02235 s
Cycle = 84 >>> Sudoku.log


   0  1  2  3  4  5  6  7  8

0  2  1  3  4  9  8  5  6  7
1  9  2  1  3  4  5  7  8  6
2  4  3  2  1  5  7  6  9  8
3  8  9  4  2  6  1  3  7  5
4  3  4  5  7  8  6  1  2  9
5  6  8  7  9  1  2  4  5  3
6  1  5  8  6  7  3  9  4  2
7  5  7  6  8  3  9  2  1  4
8  7  6  9  5  2  4  8  3  1

In [6]:
import numpy as np

nums = np.arange(100000, 100000)


In [22]:
%%timeit
to_str1 = [np.sum(x) for x in nums]

669 ns ± 2.95 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [23]:
%%timeit
to_str1 = [np.sum(x) for x in nums]

679 ns ± 1.93 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [24]:
%%timeit
to_str2 = list(map(np.sum, nums))

872 ns ± 4.55 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [25]:
%%timeit
to_str2 = list(map(np.sum, nums))

878 ns ± 4.74 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [26]:
import gc
gc.collect()

24

In [27]:
%%timeit
to_str2 = list(map(np.sum, nums))

879 ns ± 2.83 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [28]:
%%timeit
to_str2 = list(map(np.sum, nums))

872 ns ± 5.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [29]:
%%timeit
to_str1 = [np.sum(x) for x in nums]

682 ns ± 1.81 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [30]:
%%timeit
to_str1 = [np.sum(x) for x in nums]

681 ns ± 1.57 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [40]:
%%timeit
[tuple(x) for x in nums]

685 ns ± 2.52 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [41]:
%%timeit
[tuple(x) for x in nums]

683 ns ± 2.63 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [42]:
%%timeit
list(map(tuple, nums))

832 ns ± 3.48 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [43]:
%%timeit
list(map(tuple, nums))

834 ns ± 4.44 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [44]:
import gc
gc.collect()

1648

In [14]:
%%timeit
list(map(set, nums))

852 ns ± 1.29 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [13]:
%%timeit
list(map(set, nums))

854 ns ± 2.06 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [12]:
%%timeit
[set(x) for x in nums]

688 ns ± 2.64 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [11]:
%%timeit
[set(x) for x in nums]

670 ns ± 1.78 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [10]:
%%timeit
to_int1 = [int(x) for x in to_str1]

1.01 ms ± 4.83 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [11]:
%%timeit
to_int2 = list(map(int, to_str2))

758 µs ± 2.94 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [12]:
%%timeit
to_int2 = list(map(int, to_str2))

756 µs ± 5.45 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [13]:
%%timeit
to_int1 = [int(x) for x in to_str1]

986 µs ± 5.56 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
